### Authentification & initial params

In [1]:
!git clone https://github.com/TheChwal/forest_guard.git

fatal: destination path 'forest_guard' already exists and is not an empty directory.


In [ ]:
%cd forest_guard/

/content/forest_guard


In [ ]:
!pip install .

In [ ]:
pip install tensorflow-addons

In [ ]:
import ee 
import folium
from datetime import datetime as dt
from IPython.display import Image
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from google.cloud import storage
import time
import numpy as np
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import models
from tensorflow.python.keras import metrics
from tensorflow.python.keras import optimizers

In [ ]:
#TO BE DONE IN GOOGLE COLAB

from google.colab import auth

In [ ]:
auth.authenticate_user()

In [ ]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=Zro_u1IAPnAUvBqE7ybJral5pBc1l_t9qMYnLhBNIPA&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g6n6TP9WgEVd9dkM9mw5gsIGd7Dj5fbfNPMpAjsnbjCBhC9ZEXyPnI

Successfully saved authorization token.


In [ ]:
import os
from google.cloud import storage

PROJECT = 'wagon-bootcamp-data'

BUCKET = 'forest_guard_bucket'  # ⚠️ replace with your BUCKET NAME
FOLDER = 'Arthur_prediction_mont_de_marsan_with_target'
TRAINING_BASE = 'training_patches'
EVAL_BASE = 'eval_patches'

OPTICAL_BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
THERMAL_BANDS = ['B10', 'B11']
BANDS = OPTICAL_BANDS + THERMAL_BANDS
RESPONSE = 'fnf'
FEATURES = BANDS + [RESPONSE]

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]

COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Specify model training parameters.
#BATCH_SIZE = 16
# EPOCHS = 10
#BUFFER_SIZE = 2000
# OPTIMIZER = 'SGD'
# LOSS = 'MeanSquaredError'
# METRICS = ['RootMeanSquaredError']

In [ ]:
MODEL_DIR = 'gs://forest_guard_bucket/models/forest_guard/ai_platform_tversky_ownmodel_patience7'
EEIFIED_DIR = 'gs://forest_guard_bucket/models/forest_guard/eefied'
REGION = 'europe-west1'

In [ ]:
from tensorflow.python.tools import saved_model_utils


meta_graph_def = saved_model_utils.get_meta_graph_def(MODEL_DIR, 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs

# Just get the first thing(s) from the serving signature def.  i.e. this
# model only has a single input and a single output.
input_name = None
for k,v in inputs.items():
  input_name = v.name
  break

output_name = None
for k,v in outputs.items():
  output_name = v.name
  break

# Make a dictionary that maps Earth Engine outputs and inputs to 
# AI Platform inputs and outputs, respectively.
import json
input_dict = "'" + json.dumps({input_name: "array"}) + "'"
output_dict = "'" + json.dumps({output_name: "fnf"}) + "'"

In [ ]:
# You need to set the project before using the model prepare command.
!earthengine set_project {PROJECT}
!earthengine model prepare --source_dir {MODEL_DIR} --dest_dir {EEIFIED_DIR} --input {input_dict} --output {output_dict}

Instructions for updating:
non-resource variables are not supported in the long term
Successfully saved project id
Instructions for updating:
non-resource variables are not supported in the long term
Success: model at 'gs://forest_guard_bucket/models/forest_guard/eefied' is ready to be hosted in AI Platform.


In [ ]:
%%writefile config.yaml
autoScaling:
    minNodes: 10

Writing config.yaml


In [ ]:
MODEL_NAME = 'test_ai_platform_tversky_ownmodel_patience7'
VERSION_NAME = 'v' + str(int(time.time()))
print('Creating version: ' + VERSION_NAME)

!gcloud ai-platform models create {MODEL_NAME} \
  --project {PROJECT} \
  --region {REGION}

!gcloud ai-platform versions create {VERSION_NAME} \
  --project {PROJECT} \
  --model {MODEL_NAME} \
  --region {REGION} \
  --origin {EEIFIED_DIR} \
  --framework "TENSORFLOW" \
  --runtime-version 2.4 \
  --python-version 3.7 \
  --config=config.yaml

Creating version: v1622559645
Using endpoint [https://europe-west1-ml.googleapis.com/]
ERROR: (gcloud.ai-platform.models.create) Resource in projects [wagon-bootcamp-data] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name
Using endpoint [https://europe-west1-ml.googleapis.com/]
ERROR: (gcloud.ai-platform.versions.create) HttpError accessing <https://europe-west1-ml.googleapis.com/v1/projects/wagon-bootcamp-data/models/test_ai_platform_tversky_ownmodel_patience7/versions?alt=json>: response: <{'vary': 'Origin, X-Origin, Referer', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'date': 'Tue, 01 Jun 2021 15:01:20 GMT', 'server': 'ESF', 'cache-control': 'private', 'x-xss-protection': '0', 'x-frame-options': 'SAMEORIGIN', 'x-content-type-options': 'nosniff', 'tr

In [ ]:
# Use Landsat 8 surface reflectance data.
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

# Cloud masking function.
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask1 = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  mask2 = image.mask().reduce('min')
  mask3 = image.select(OPTICAL_BANDS).gt(0).And(
          image.select(OPTICAL_BANDS).lt(10000)).reduce('min')
  mask = mask1.And(mask2).And(mask3)
  return image.select(OPTICAL_BANDS).divide(10000).addBands(
          image.select(THERMAL_BANDS).divide(10).clamp(273.15, 373.15)
            .subtract(273.15).divide(100)).updateMask(mask)

# The image input data is a cloud-masked median composite.
image = l8sr.filterDate(
    '2015-01-01', '2017-12-31').map(maskL8sr).median().select(BANDS).float()

# Load the trained model and use it for prediction.  If you specified a region 
# other than the default (us-central1) at model creation, specify it here.
model = ee.Model.fromAiPlatformPredictor(
    projectId = PROJECT,
    modelName = MODEL_NAME,
    # modelName = "ai_platform_ee_output_format_tf2",
    version = VERSION_NAME,
    # version = "v1622550169",
    region= REGION,
    inputTileSize = [256, 256],
    # inputOverlapSize = [8, 8],
    proj = ee.Projection('EPSG:4326').atScale(30),
    fixInputProj = True,
    outputBands = {'fnf': {
        'type': ee.PixelType.float()
      }
    }
)
predictions = model.predictImage(image.toArray())

# Use folium to visualize the input imagery and the predictions.
# mapid = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
map = folium.Map(location=[49, 7.5], zoom_start=15)
# folium.TileLayer(
#     tiles=mapid['tile_fetcher'].url_format,
#     attr='Google Earth Engine',
#     overlay=True,
#     name='median composite',
#   ).add_to(map)

mapid = predictions.getMapId({'min': 0, 'max': 1})

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='predictions',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [ ]:
prediction = model.predictImage(image)

In [ ]:
prediction.getInfo()

In [ ]:
image.getInfo()

In [ ]:
tilfetch = mapid["tile_fetcher"]

# New section

In [ ]:
tilfetch.url_format

'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/00cb15470a194a6ff75f17f33efe8a6e-301ad53b983a161ecdcff33d2444fa47/tiles/{z}/{x}/{y}'